In [1]:
import requests
import json
import dotenv
import os
from groq import Groq

dotenv.load_dotenv()

OPENROUTER_API_KEY = os.environ["OPENROUTER_KEY"]
GROQ_KEY = os.environ["GROQ_KEY"]

client = Groq(
    api_key=os.environ.get("GROQ_KEY")
)
def get_response(message):
    chat_completion = client.chat.completions.create(
        messages = [
            {
                "role": "user",
                "content": f"{message}"
            }
        ],
        model=  "llama3-70b-8192"
    )
    
    return chat_completion


response= get_response("1+ 3+ 5 = ")

In [2]:
print(response.choices[0].message.content)

Let's add those numbers up!

1 + 3 = 4
4 + 5 = 9

So, 1 + 3 + 5 = 9!


In [4]:
import json
with open("gpt-crawler/output-1.json", "r") as f:
    data = json.load(f)

In [6]:
content_map = {}
titles = []
for topic in data:
    titles.append(topic['title'])
    content_map[topic['title']] = topic['html']
        

In [29]:
def templater(title, content):
    return f"""
    Given the following documentation in HTML, please first summarize step by step what the section is about and then provide a code snippet that demonstrates the functionality of the section.
    ################################################
    You are given the title: {title}
    ################################################
    Here is the documentation:
    {content}
    """
    


def ex_templater(summary):
    return f"""
    Given the following summary, please write a coding exercise for the user to complete. You should write most of the code out already, and only leave a few lines blank for the user to write out
    
    You should only generate the instructions and the code exercise. The solutions and explanations are to be omitted. The focus on the exercises/incomplete lines should be on the code related to the given documentation
    
    The following example is an example of what you should output:
   Here are a few coding exercises based on the summary:

################################################
Exercise 1:
Write code to compute the inverse discrete Fourier transform of a 1D signal using `torch.fft.ifft()`.

```python
import torch

# Create a sample signal
signal = torch.tensor([1, 2, 3, 4, 5, 6, 7, 8])

# Compute the discrete Fourier transform of the signal
fft_signal = torch.fft.fft(signal)

###TODO: Compute the inverse discrete Fourier transform of the signal (roughly 1 line)

print(ifft_signal)
```

################################################
Exercise 2:
Write code to compute the 2D discrete Fourier transform of a 2D signal using `torch.fft.fftn()`.

```python
import torch

# Create a sample 2D signal
signal = torch.tensor([[1, 2, 3], [4, 5, 6], [7, 8, 9]])

###TODO: Compute the 2D discrete Fourier transform of the signal (roughly 1 line)

print(fftn_signal)
```

################################################
Exercise 3:
Write code to shift the zero-frequency component of a 1D signal to the center of the spectrum using `torch.fft.fftshift()`.

```python
import torch

# Create a sample signal
signal = torch.tensor([1, 2, 3, 4, 5, 6, 7, 8])

# Compute the discrete Fourier transform of the signal
fft_signal = torch.fft.fft(signal)

###TODO: Shift the zero-frequency component of the signal to the center of the spectrum (roughly 1 line)

print(shifted_fft_signal)
```

Note: You need to complete the exercises by filling in the `###TODO` lines with the appropriate code.
    Here is the summary:
    {summary}
    """


for i in titles[:10]:
    topic_ex = content_map[i]
    template = templater(i, topic_ex)
    res = get_response(template)
    
    summary = res.choices[0].message.content
    ex_template = ex_templater(summary)
    ex = get_response(ex_template)
    print(ex.choices[0].message.content)
    
    break

exercise = ex.choices[0].message.content

Here are a few coding exercises based on the summary:

################################################
Exercise 1:
Write code to compute the 2D discrete Fourier transform of a 2D signal using `torch.fft.fftn()`.

```python
import torch

# Create a sample 2D signal
signal = torch.tensor([[1, 2, 3], [4, 5, 6], [7, 8, 9]])

###TODO: Compute the 2D discrete Fourier transform of the signal (roughly 1 line)

print(fftn_signal)
```

################################################
Exercise 2:
Write code to shift the zero-frequency component of a 1D signal to the center of the spectrum using `torch.fft.fftshift()`.

```python
import torch

# Create a sample signal
signal = torch.tensor([1, 2, 3, 4, 5, 6, 7, 8])

# Compute the discrete Fourier transform of the signal
fft_signal = torch.fft.fft(signal)

###TODO: Shift the zero-frequency component of the signal to the center of the spectrum (roughly 1 line)

print(shifted_fft_signal)
```

################################################
Exercise

In [30]:
def answer_template(exercise):
    return f"""
    Given the following coding exercise, please write out the code that would complete the exercise. You should write out the code that would complete the exercise.

    Example solution will be:
    Here are the completed exercises with test cases:

**Exercise 1**
```python
import torch
import numpy as np

# Create a sample signal
signal = torch.tensor([1, 2, 3, 4, 5, 6, 7, 8], dtype=torch.float32)

# Compute the sample frequencies associated with the Fourier transform of the signal
freq = torch.fft.fftfreq(signal.size()[0], dtype=torch.float32)

print(freq)

**Testcase 1**
assert freq.shape == torch.Size([8])
assert torch.allclose(freq, torch.tensor([-0.5, -0.375, -0.25, -0.125, 0.125, 0.25, 0.375, 0.5]))
```

**Exercise 2**
```python
import torch

# Create a sample 2D signal
signal = torch.tensor([[1, 2, 3], [4, 5, 6], [7, 8, 9]])

# Compute the 2D discrete Fourier transform of the signal
fftn_signal = torch.fft.fftn(signal)

# Reorder the elements of the FFT data
reordered_fftn_signal = torch.fft.fftshift(fftn_signal)

print(reordered_fftn_signal)

**Testcase 2**
assert reordered_fftn_signal.shape == torch.Size([3, 3])
assert torch.allclose(reordered_fftn_signal, torch.tensor([[7.06225781e+01+1.22464680e-16j, 2.12132034e+00-1.22464680e-16j, 2.12132034e+00+1.22464680e-16j], 
                                                            [2.12132034e+00-1.22464680e-16j, 3.53553391e+00+1.22464680e-16j, 3.53553391e+00-1.22464680e-16j], 
                                                            [2.12132034e+00+1.22464680e-16j, 3.53553391e+00-1.22464680e-16j, 7.06225781e+01-1.22464680e-16j]]))
```

**Exercise 3**
```python
import torch

# Create a sample 2D signal
signal = torch.tensor([[1, 2, 3], [4, 5, 6], [7, 8, 9]])

# Compute the 2D discrete Fourier transform of the signal
fftn_signal = torch.fft.fftn(signal)

# Compute the inverse 2D discrete Fourier transform of the signal
ifftn_signal = torch.fft.ifftn(fftn_signal)

print(ifftn_signal)

**Testcase 3**
assert ifftn_signal.shape == torch.Size([3, 3])
assert torch.allclose(ifftn_signal, signal)
```
    
    Please also generate test cases for the code that you write. You should ensure that the testcases work. You need to make sure other than the incomplete line, the rest of the previous template code is unchanged. 
    
    {exercise}

"""

ans_temp = answer_template(ex)
ans = get_response(ans_temp)
print(ans.choices[0].message.content)

Here are the completed exercises with test cases:

**Exercise 1**
```python
import torch

# Create a sample 2D signal
signal = torch.tensor([[1, 2, 3], [4, 5, 6], [7, 8, 9]])

# Compute the 2D discrete Fourier transform of the signal
fftn_signal = torch.fft.fftn(signal)

print(fftn_signal)

**Testcase 1**
assert fftn_signal.shape == torch.Size([3, 3])
assert torch.allclose(fftn_signal, torch.tensor([[21.+0.j, -3.+3.j, -3.-3.j], 
                                                  [-3.+3.j, -3.+1.j, -1.-1.j], 
                                                  [-3.-3.j, -1.-1.j, 21.+0.j]]))
```

**Exercise 2**
```python
import torch

# Create a sample signal
signal = torch.tensor([1, 2, 3, 4, 5, 6, 7, 8])

# Compute the discrete Fourier transform of the signal
fft_signal = torch.fft.fft(signal)

# Shift the zero-frequency component of the signal to the center of the spectrum
shifted_fft_signal = torch.fft.fftshift(fft_signal)

print(shifted_fft_signal)

**Testcase 2**
assert shifted_fft_si

In [ ]:
import nbformat
from nbformat import v4 as nbf


# Initialize lists to store extracted exercises, solutions, and test cases
exercises = []
solutions = []
test_cases = []
exercise_pattern = r"Exercise\s\d+:(.*?)\n```"
solution_pattern = r"```python\n(.*?)\n```"
testcase_pattern = r"\*\*Testcase\s\d+\*\*\n```\n(.*?)\n```\n"


t